In [ ]:
# ------------------------------ cleaned and refined for xl.---------
import pandas as pd
import re
import string
import spacy
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment

# Load spaCy English model
nlp = spacy.load("en_core_web_sm", disable=["ner", "parser"])

# Load the dataset
df = pd.read_csv('job_title_des.csv')

# Step 1: Basic text cleaning
def clean_text(text):
    if pd.isnull(text):
        return ''
    text = text.lower()
    text = re.sub(r'http\S+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.encode('ascii', 'ignore').decode()
    return text

# Step 2: spaCy lemmatization and stopword removal
def spacy_preprocess_texts(texts):
    processed_texts = []
    for doc in nlp.pipe(texts, batch_size=1000, n_process=-1):
        tokens = [
            token.lemma_ for token in doc
            if not token.is_stop and not token.is_punct
            and not token.is_space and token.is_alpha and len(token) > 2
        ]
        processed_texts.append(' '.join(tokens))
    return processed_texts

# Cleaning
df['Job Title'] = df['Job Title'].astype(str).apply(clean_text)
df['Job Description'] = df['Job Description'].astype(str).apply(clean_text)

# Combine duplicates
df = df.groupby('Job Title', as_index=False).agg({'Job Description': ' '.join})

# Preprocess job descriptions
df['Job Description'] = spacy_preprocess_texts(df['Job Description'].tolist())

# Drop empty rows, imputation
df = df[df['Job Description'].str.strip().astype(bool)]

# Save to Excel (instead of CSV for better readability)
excel_path = 'cleaned_job_data.xlsx'
df.to_excel(excel_path, index=False)

# Format Excel (wrap text + auto column width)
wb = load_workbook(excel_path)
ws = wb.active

# Apply wrap text and auto column width
for col in ws.columns:
    max_length = 0
    col_letter = get_column_letter(col[0].column)
    for cell in col:
        cell.alignment = Alignment(wrap_text=True)
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))
    ws.column_dimensions[col_letter].width = min(max_length, 60)

wb.save(excel_path)

print(f"Structured and formatted Excel file saved to '{excel_path}'")


Structured and formatted Excel file saved to 'cleaned_job_data.xlsx'


In [ ]:
# ------metrics, based on how well preprocessing takes place--------
import pandas as pd
import re
from collections import Counter

# Load before and after datasets
raw_df = pd.read_csv('job_title_des.csv')                      # Original unclean data
# read the xl file instead of csv
clean_df = pd.read_excel('cleaned_job_data.xlsx')              # After preprocessing store it in a .xlsx for a neater format

# 1. Count duplicates removed
raw_titles = raw_df['Job Title'].nunique()
clean_titles = clean_df['Job Title'].nunique()
print(f"Unique Job Titles: Before = {raw_titles}, After = {clean_titles}")

# 2. Reduction in rows
print(f"Total Rows: Before = {len(raw_df)}, After = {len(clean_df)}")

# 3. Average description length
raw_avg_len = raw_df['Job Description'].astype(str).apply(lambda x: len(x.split())).mean()
clean_avg_len = clean_df['Job Description'].astype(str).apply(lambda x: len(x.split())).mean()
print(f"Avg Job Description Length: Before = {raw_avg_len:.2f}, After = {clean_avg_len:.2f}")

# 4. Top 10 most common lemmatized words after preprocessing
all_words = ' '.join(clean_df['Job Description']).split()
top_words = Counter(all_words).most_common(10)
print("Top 10 Lemmatized Words:", top_words)

# 5. % of rows removed due to empty descriptions
removed_due_to_empty = len(raw_df) - len(clean_df)
print(f"Rows with empty/removed descriptions: {removed_due_to_empty} ({(removed_due_to_empty/len(raw_df))*100:.2f}%)")


Unique Job Titles: Before = 15, After = 15
Total Rows: Before = 2277, After = 15
Avg Job Description Length: Before = 276.20, After = 4014.00
Top 10 Lemmatized Words: [('experience', 1572), ('work', 1001), ('year', 727), ('development', 686), ('application', 513), ('design', 513), ('knowledge', 499), ('team', 491), ('skill', 484), ('software', 427)]
Rows with empty/removed descriptions: 2262 (99.34%)


In [ ]:
# used sonar, and got o/p
import pandas as pd
import requests
import time

# Load your Excel file
df = pd.read_excel("cleaned_job_data.xlsx")

jd_column = "Job Description"

# API setup, LLM magic starts here if you are using perplexity
url = "https://api.perplexity.ai/chat/completions"
headers = {
    "Authorization": "Bearer YOUR_API_KEY",
    "Content-Type": "application/json"
}

# Function to query Perplexity API using sonar
def generate_curriculum(jd_text):
    payload = {
        "model": "sonar",
        "messages": [
            {
                "role": "system",
                "content": "You are a precise and concise career coach and curriculum designer."
            },
            {
                "role": "user",
                "content": f"""
Given the following job description, create a detailed but concise 6-week learning curriculum to help someone become job-ready for this role. Include week-wise topics, key concepts, and 1-2 top resources per week (free if possible).

Job Description:
{jd_text}
"""
            }
        ]
    }

    response = requests.post(url, json=payload, headers=headers)

    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        print(f"Error {response.status_code}: {response.text}")
        return f"ERROR: {response.status_code}"

# Generate for each JD and collect results
results = []

for idx, row in df.iterrows():
    jd = row[jd_column]
    try:
        print(f"Processing JD #{idx + 1}")
        curriculum = generate_curriculum(jd)
        results.append({
            "job_description": jd,
            "curriculum": curriculum
        })
        time.sleep(1)  # throttle to avoid hitting rate limits, makes it sleep after each iteration to prevent rate limiting
    except Exception as e:
        results.append({
            "job_description": jd,
            "curriculum": f"ERROR: {str(e)}"
        })

# Save results to Excel
pd.DataFrame(results).to_excel("generated_curriculums_sonar.xlsx", index=False)
print("Curriculums saved to generated_curriculums_sonar.xlsx")


Processing JD #1
Processing JD #2
Processing JD #3
Processing JD #4
Processing JD #5
Processing JD #6
Processing JD #7
Processing JD #8
Processing JD #9
Processing JD #10
Processing JD #11
Processing JD #12
Processing JD #13
Processing JD #14
Processing JD #15
Curriculums saved to generated_curriculums_sonar.xlsx


In [ ]:
# load for metrics evaluation particularly rouge score+nltk
!pip install pandas nltk rouge-score bert-score openpyxl

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.9 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=2

In [ ]:
# for chrf++ metric
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score
from sacrebleu.metrics import CHRF
import warnings

warnings.filterwarnings("ignore")

# Load the Excel file
df = pd.read_excel("updated_gold_curriculums.xlsx")

# Drop rows with missing reference
df = df.dropna(subset=["gold_std_cirriculi", "curriculum"])

# BLEU setup
smoothie = SmoothingFunction().method4
def compute_bleu(reference, prediction):
    return sentence_bleu([reference.split()], prediction.split(), smoothing_function=smoothie)

# ROUGE setup
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
def compute_rougeL(reference, prediction):
    scores = scorer.score(reference, prediction)
    return scores["rougeL"].fmeasure

# CHRF++ setup
chrf = CHRF(word_order=2)  # word_order=2 enables chrF++ instead of plain chrF
def compute_chrf(reference, prediction):
    return chrf.sentence_score(prediction, [reference]).score

# Compute BLEU, ROUGE-L, CHRF++
df["bleu_score"] = df.apply(lambda row: compute_bleu(row["gold_std_cirriculi"], row["curriculum"]), axis=1)
df["rougeL"] = df.apply(lambda row: compute_rougeL(row["gold_std_cirriculi"], row["curriculum"]), axis=1)
df["chrf++"] = df.apply(lambda row: compute_chrf(row["gold_std_cirriculi"], row["curriculum"]), axis=1)

# Compute BERTScore(can take time)
print("Computing BERTScore...")
P, R, F1 = score(df["curriculum"].tolist(), df["gold_std_cirriculi"].tolist(), lang="en", verbose=True)
df["bertscore_precision"] = P.tolist()
df["bertscore_recall"] = R.tolist()
df["bertscore_f1"] = F1.tolist()

# Save results
df.to_excel("evaluated_curriculums.xlsx", index=False)
print("Evaluation complete. Saved to evaluated_curriculums.xlsx")


Computing BERTScore...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 143.05 seconds, 0.10 sentences/sec
Evaluation complete. Saved to evaluated_curriculums.xlsx
